In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [8]:
company ='NIO'

start=dt.datetime(2000,1,1)
end=dt.datetime(2021,10,27)

data=web.DataReader(company,'yahoo',start,end)

scalar =MinMaxScaler(feature_range=(0,1))
scaled_data=scalar.fit_transform(data['Close'].values.reshape(-1,1))
prediction_days=360
x_train=[]
y_train=[]

for x in range(prediction_days,len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x,0])
    y_train.append(scaled_data[x,0])
    
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

model=Sequential()

model.add(LSTM(units=50,return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
          
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train,y_train,epochs=10,batch_size=32)

test_start=dt.datetime(2021,1,1)
test_end=dt.datetime.now()
          
test_data=web.DataReader(company,'yahoo',test_start,test_end)
actual_price=test_data['Close'].values
total_dataset=pd.concat((data['Close'],test_data['Close']),axis=0)
          
model_inputs=total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
model_inputs=model_inputs.reshape(-1,1)
model_inputs=scalar.transform(model_inputs)
          
#make prediction
# x_test=[]
# for x in range(prediction_days,len(model_inputs)):
#           x_test.append(model_inputs[x-prediction_days:x,0])

# x_test=np.array(x_test)
# x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
          
# prediction_prices=model.predict(x_test)
# prediction_prices=scalar.inverse_transform(prediction_prices)
          
# plt.plot(actual_price, color="black", label=f"Actual {company} price")
# plt.plot(prediction_prices,color='green',label=f"Predicted {company} Price")
# plt.title(f"{company} Share Price")
# plt.xlabel('Time')
# plt.ylabel(f'{company} Share Price')
# plt.legend()
# plt.show()

# predict for tomorrow
real_data=[model_inputs[len(model_inputs)+1-prediction_days:len(model_inputs+1),0]]
real_data=np.array(real_data)
real_data=np.reshape(real_data,(real_data.shape[0],real_data.shape[1],1))

prediction=model.predict(real_data)
prediction=scalar.inverse_transform(prediction)
print(f"Prediction: {prediction}")

Epoch 1/10
14/14 [==============================] - 8s 360ms/step - loss: 0.0954
Epoch 2/10
14/14 [==============================] - 5s 382ms/step - loss: 0.0201
Epoch 3/10
14/14 [==============================] - 5s 375ms/step - loss: 0.0104
Epoch 4/10
14/14 [==============================] - 5s 385ms/step - loss: 0.0102
Epoch 5/10
14/14 [==============================] - 6s 432ms/step - loss: 0.0079
Epoch 6/10
14/14 [==============================] - 6s 448ms/step - loss: 0.0084
Epoch 7/10
14/14 [==============================] - 6s 439ms/step - loss: 0.0072
Epoch 8/10
14/14 [==============================] - 7s 467ms/step - loss: 0.0078
Epoch 9/10
14/14 [==============================] - 6s 441ms/step - loss: 0.0077
Epoch 10/10
14/14 [==============================] - 8s 555ms/step - loss: 0.0068
Prediction: [[37.65421]]
